# ML-7 Оптимизация гиперпараметров модели
###  Содержание <a class="anchor" id=0></a>
- [1. Введение](#1)
- [2. Базовая оптимизация](#2)
- [3. Продвинутая оптимизация](#3)
- [4. Практика](#4)
- [5. Итоги](#5)


# 1. Введение <a class="anchor" id=1></a>

[к содержанию](#0)

Итак, как мы уже неоднократно упоминали ранее, в машинном обучении есть два типа параметров.

**Внутренние** (параметры модели)

Подбираются во время обучения и определяют, как использовать входные данные для получения необходимого результата.

Например, это веса (коэффициенты уравнения) в линейной/логистической регрессии.

**Внешние** (параметры алгоритма)

Их принято называть `гиперпараметрами`. Внешние параметры могут быть произвольно установлены перед началом обучения и контролируют внутреннюю работу обучающего алгоритма.

Например, это параметр регуляризации в линейной/логистической регрессии.

`Гиперпараметры` отвечают за сложность взаимосвязи между входными признаками и целевой переменной, поэтому сильно влияют на модель и качество прогнозирования.

Продемонстрируем это на примере задачи регрессии с помощью двух графиков работы алгоритма случайного леса, построенного на основе `5`, `100` деревьев (`n_estimators = [5, 100]`):

<img src=ml7_img1.png>

Видим, что при 100 деревьях модель находит более сложную закономерность в данных и точность соответственно будет выше, чем при 5.

Каждый алгоритм МО имеет набор гиперпараметров, которые определяют, как именно он строит модель на обучающей выборке. Например, в модуле ML-2 для повышения эффективности модели мы уже рассматривали подбор параметра регуляризации `alpha` для алгоритма линейной регрессии `Ridge`.

In [1]:
import numpy as np
import matplotlib as plt
from sklearn import linear_model
from sklearn import metrics

In [2]:
#Создаем список из 20 возможных значений от 0.001 до 10
alpha_list = np.linspace(0.01, 10, 20)
#Создаем пустые списки, в которые будем добавлять результаты 
train_scores = []
test_scores = []
for alpha in alpha_list:
    #Создаем объект класса линейная регрессия с L2-регуляризацией
    ridge_lr_poly = linear_model.Ridge(alpha=alpha, max_iter=10000)
    #Обучаем модель предсказывать логарифм целевого признака
    ridge_lr_poly.fit(X_train_scaled_poly, y_train_log)
    #Делаем предсказание для каждой из выборок
    #Если обучили на логарифме, то от результата необходимо взять обратную функцию - экспоненту
    y_train_predict_poly = np.exp(ridge_lr_poly.predict(X_train_scaled_poly))
    y_test_predict_poly = np.exp(ridge_lr_poly.predict(X_test_scaled_poly))
    #Рассчитываем метрику для двух выборок и добавляем их в списки
    train_scores.append(metrics.mean_absolute_error(y_train, y_train_predict_poly))
    test_scores.append(metrics.mean_absolute_error(y_test, y_test_predict_poly))
 
#Визуализируем изменение R^2 в зависимости от alpha
fig, ax = plt.subplots(figsize=(12, 4)) #фигура + координатная плоскость
ax.plot(alpha_list, train_scores, label='Train') #линейный график для тренировочной выборки
ax.plot(alpha_list, test_scores, label='Test') #линейный график для тестовой выборки
ax.set_xlabel('Alpha') #название оси абсцисс
ax.set_ylabel('MAE') #название оси ординат
ax.set_xticks(alpha_list) #метки по оси абцисс
ax.xaxis.set_tick_params(rotation=45) #поворот меток на оси абсцисс
ax.legend(); #отображение легенды

NameError: name 'X_train_scaled_poly' is not defined

<img src=ml7_img2.png>

Наилучшее значение метрики соответствует `alpha = 0.01` (кстати, можно попробовать перебрать значения `alpha < 0.01`).

В данном случае мы просто воспользовались циклом for и перебрали некоторые заданные значения alpha, хотя, по всей видимости, не самые оптимальные. Поэтому подобранные эмпирическим путём значения гиперпараметров с большей вероятностью дадут низкую прогностическую эффективность.

Также рассмотренный метод визуализации зависимости метрики от гиперпараметра позволяет выбрать только один внешний параметр, в данном случае — `alpha`. А что делать, если у нас не один, а несколько? 

Например, вспомним основные внешние параметры `DecisionTreeClassifier`:

* `criterion` — критерий информативности. Может быть равен `'gini'` — критерий Джини — и `'entropy'` — энтропия Шеннона.
* `max_depth` — максимальная глубина дерева. По умолчанию `None`, глубина дерева не ограничена.
* `max_features` — максимальное число признаков, по которым ищется лучшее разбиение в дереве. По умолчанию `None`, то есть обучение производится на всех признаках.
* `min_samples_leaf` — минимальное число объектов в листе. По умолчанию — `1`.

Мы, конечно, можем сделать кучу вложенных циклов. Однако, поскольку поиск оптимальных значений гиперпараметров является общераспространенной задачей МО, библиотека `scikit`-`learn` и другие предлагают методы, позволяющие её решить.

Тщательный подбор гиперпараметров гарантирует, что модель покажет максимально возможную точность на обучающих данных, **но это совершенно не означает хороший результат на тестовых или новых данных.**

Поиск оптимальных значений гиперпараметров модели является сложной задачей, обязательной почти для всех моделей и наборов данных. Однако важно понимать смысл гиперпараметров перед их подбором.

# 2. Базовая оптимизация <a class="anchor" id=2></a>

[к содержанию](#0)

В базовой оптимизации, предоставляемой библиотекой `sklearn`, есть два основных метода — `grid search` и `random search`. С ними мы сейчас и познакомимся. Оба используются при решении реальных задач, поэтому важно разобраться, как они устроены. 

Наиболее часто используемый метод — это **поиск по сетке** (`grid search`), который по сути является попыткой перебрать все возможные комбинации заданных гиперпараметров. Мы указываем список значений для различных гиперпараметров, и, ориентируясь на нашу метрику, оцениваем эффективность модели для каждого их сочетания, чтобы получить оптимальную комбинацию значений.

Допустим, мы хотим подобрать гиперпараметры `min_samples_leaf` и `max_depth` для алгоритма `DecisionTreeClassifier`. Зададим списки их значений:

In [ ]:
min_samples_leaf = [3, 5, 8, 9]
max_depth = [4, 5, 6, 7, 8]

Поскольку нам нужно перебрать четыре различных значения для `min_samples_leaf` и пять — для `max_depth`, то получается всего `4*5=20` комбинаций. Модель будет обучена 20 раз; столько же раз будет рассчитана метрика.

Сетка выглядит следующим образом:

<img src=ml7_img3.png>

# ОПАСНОСТЬ ПЕРЕОБУЧЕНИЯ И УТЕЧКИ ДАННЫХ

Для того, чтобы выбрать оптимальные значения гиперпараметров, мы ориентируемся на выбранную метрику, рассчитанную на тестовой выборке. Мы делали это для подбора гиперпараметра регуляризации `alpha`, но является ли это надёжным подходом?

>Эту проблему мы уже обсуждали в модуле ML-5 «Валидация и оценка качества моделей».

Давайте вспомним: мы перебираем множество значений гиперпараметров и выбираем ту комбинацию значений, которая даёт наилучшую точность на тестовых данных. **Однако это совсем не означает, что на новых данных мы получим такой же результат**. 

Поскольку мы использовали тестовый набор для настройки гиперпараметров, мы больше не можем использовать его для оценки качества модели. Теперь в этих целях нам необходим независимый набор данных, то есть набор, который не использовался для построения модели и настройки её гиперпараметров.

Следовательно, надо разбить данные на **три части**: **обучающую** для построения модели, проверочную (**валидационную**) для выбора гиперпараметров модели, а также **тестовую** для оценки качества модели и выбранных гиперпараметров. 

<img src=ml7_img4.png>

Наличие всех трёх наборов данных критически важно для использования МО. Любой подбор гиперпараметров, сделанный на тестовых данных, «сливает» модели информацию, содержащуюся в них, и может привести к неправильной оценке качества модели. Такая проблема относится к категории утечки данных, которую мы уже тоже затрагивали в модуле по валидации.

>Рассмотренный метод разбиения данных на обучающий, проверочный и тестовый наборы является вполне рабочим и относительно широко используемым, но весьма чувствителен к равномерности разбиения данных. 

Для лучшей оценки обобщающей способности вместо одного разбиения данных на обучающий и проверочный наборы мы можем воспользоваться перекрёстной проверкой, то есть **кросс-валидацией** (`cross` `validation`). В таком случае качество модели оценивается для каждой комбинации гиперпараметров по всем разбиениям кросс-валидации. 

<img src=ml7_img5.png>

>Пояснение к рисунку. Предположим, что у нас есть `n` комбинаций гиперпараметров. Берём первую комбинацию и обучаем на них первую модель с помощью кросс-валидации с 10 фолдами (`cv=10`), затем рассчитываем метрику как среднее по всем разбиениям. Так проделываем для каждой комбинации и выбираем ту, при которой наша метрика наилучшая. В итоге мы обучим `n*cv` моделей, но выберем один набор гиперпараметров, который и будет использоваться для обучения итоговой модели на всей обучающей выборке.

# GRIDSEARCHCV

Поскольку поиск по сетке с кросс-валидацией является весьма распространённым методом настройки гиперпараметров, библиотека `scikit`-`learn` предлагает класс `GridSearchCV`, в котором осуществляется именно такой вариант.

## Файл `ML-7.Optimization_of_hyperparameters.ipynb`



# 3. Продвинутая оптимизация <a class="anchor" id=3></a>

[к содержанию](#0)

# 4. Практика <a class="anchor" id=4></a>

[к содержанию](#0)

# 5. Итоги <a class="anchor" id=5></a>

[к содержанию](#0)